In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import findLLJ

# NYSERDA Data
Downloaded from https://oswbuoysny.resourcepanorama.dnv.com/. CSV files are NOT included in this repository and should be downloaded and saved by the user.

In [3]:
# Replace with appropriate data paths

Buoy = "E06"

if Buoy == "E05":
    lidar = pd.read_csv('/Users/edejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E05_Hudson_North_10_min_avg_20190812_20210919.csv')
    hourly = pd.read_csv('/Users/edejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E05_Hudson_North_hourly_avg_20190812_20210919.csv')
elif Buoy == "E06":
    lidar = pd.read_csv('/Users/edejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E06_Hudson_South_10_min_avg_20190904_20220327.csv')
    hourly = pd.read_csv('/Users/edejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E06_Hudson_South_hourly_avg_20190904_20220327.csv')

/Users/edejong/opt/anaconda3/envs/mmc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,60,62,63,64,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
lidar = lidar[0:-5]

### Create netcdf file with raw lidar information

In [5]:
waterT = pd.DataFrame(pd.to_numeric(hourly[' ADCP_ADCPtemp'], errors='coerce'))
waterT['Datetime'] = pd.to_datetime(hourly['timestamp'], errors='coerce')
waterT = waterT.set_index('Datetime')
upsampledT = waterT.resample('10min')
interpolatedT = upsampledT.interpolate(method='linear')

In [6]:
lidar['Datetime'] = pd.to_datetime(lidar['timestamp'], errors='coerce')
lidar = lidar.set_index('Datetime')

In [7]:
lidarWaterT = interpolatedT.loc[lidar.index]

In [8]:
alts = ['18m','38m','58m','78m','98m','118m','138m','158m','178m','198m']
lidar_ws_keys = [] #['timestamp']
lidar_wd_keys = [] #['timestamp']
for alt in alts:
    lidar_ws_keys.append(' lidar_lidar'+alt+'_Z10_HorizWS')
    lidar_wd_keys.append(' lidar_lidar'+alt+'_WD_alg_03')

In [9]:
windspeeds=np.asarray(lidar[lidar_ws_keys], dtype='float64')
winddirections=np.asarray(lidar[lidar_wd_keys], dtype='float64')
datetimes=np.asarray(pd.to_datetime(lidar['timestamp']))
heights = [18, 38, 58, 78, 98, 118, 138, 158, 178, 198]

In [10]:
ta=np.asarray(lidar[" meteo_Ta_avg"], dtype="float64")
pa=np.asarray(lidar[" meteo_Pa_avg"], dtype="float64")
rh=np.asarray(lidar[" meteo_Ua_avg"], dtype="float64")
waterT=np.asarray(lidarWaterT[" ADCP_ADCPtemp"], dtype="float64")

In [11]:
lidar_xr = xr.Dataset(
    data_vars = dict(
        windspeed=(["t","z"], windspeeds),
        winddirection=(["t","z"], winddirections),
        temp=(["t"], ta),
        pressure=(["t"], pa),
        rel_humidity=(["t"], rh),
        waterT=(["t"], waterT)
    ),
    coords=dict(
        datetime=(["t"], datetimes),
        height=(["z"], heights)
    )
)

In [12]:
def u_v_from_WS_WD(WS, WD):
    u = WS * np.cos((270 - WD)/180 * np.pi)
    v = WS * np.sin((270 - WD)/180 * np.pi)
    return (u, v)

In [13]:
(u, v) = u_v_from_WS_WD(lidar_xr['windspeed'], lidar_xr['winddirection'])
lidar_xr['U'] = u
lidar_xr['V'] = v

In [14]:
lidar_xr.to_netcdf('LiDAR/'+Buoy+'_lidar_full.nc')

In [15]:
if Buoy == 'E06':
    springtime = lidar_xr.isel(t = slice(29896, 42419))
elif Buoy == 'E05':
    springtime = lidar_xr.isel(t = slice(33552, 46657))
springtime

<xarray.Dataset>
Dimensions:        (t: 12523, z: 10)
Coordinates:
    datetime       (t) datetime64[ns] 2020-04-01 ... 2020-06-30T23:50:00
    height         (z) int64 18 38 58 78 98 118 138 158 178 198
Dimensions without coordinates: t, z
Data variables:
    windspeed      (t, z) float64 8.687 8.659 8.704 8.776 ... 1.135 1.133 1.178
    winddirection  (t, z) float64 81.16 79.44 81.96 81.17 ... 170.8 169.8 173.3
    temp           (t) float64 5.63 5.624 5.686 5.67 ... 22.41 22.5 22.5 22.44
    pressure       (t) float64 1.014e+03 1.014e+03 ... 1.012e+03 1.012e+03
    rel_humidity   (t) float64 70.9 70.54 70.93 71.31 ... 75.07 76.01 76.59
    waterT         (t) float64 8.29 8.293 8.297 8.3 ... 23.82 23.81 23.81 23.8
    U              (t, z) float64 -8.584 -8.513 -8.618 ... -0.2007 -0.1366
    V              (t, z) float64 -1.336 -1.587 -1.218 ... 1.121 1.115 1.17

In [16]:
springtime.to_netcdf('LiDAR/springtime_'+Buoy+'.nc')

### QC & Jets

In [17]:
def is_data_pt_valid(WS, WD):
    valid = True
    if (WD < 0.0).any():
        valid = False
    elif (WD > 359.99).any():
        valid = False
    elif (WS < 0.0).any():
        valid = False
    elif (WS > 70.0).any():
        valid = False
    elif np.count_nonzero(np.isnan(WS)) > 7:
        valid = False
    return valid

In [76]:
validity = np.zeros_like(lidar_xr['temp'], dtype='bool')
for i in range(len(validity)):
    validity[i] = is_data_pt_valid(lidar_xr['windspeed'].isel(t=i), lidar_xr['winddirection'].isel(t=i))

In [77]:
jet_id = findLLJ.findLLJevents_xr_lidar(lidar_xr)

In [78]:
jet_id = jet_id.rename_dims({'Time': 't'})
jet_id = jet_id.assign(dict(is_valid=(["t"], validity)))
jet_id['datetime'] = lidar_xr['datetime']
jet_id.to_netcdf('LiDAR/full_'+Buoy+'_LLJs.nc')

In [34]:
#jet_id = xr.load_dataset('LiDAR/full_'+Buoy+'_LLJs.nc')
jet_id = jet_id.assign_coords(t=jet_id['datetime'])
spring_jets = jet_id.sel(t=slice('2020-04-01', '2020-07-01'))
spring_jets.to_netcdf('LiDAR/springtime_'+Buoy+'_LLJs_2020.nc')

# SGP Data
Downloaded from https://www.arm.gov/capabilities/instruments/dl for SGP site C1, 2018-06-08 through 2018-06-20. NetCDF file is included here as well.

In [8]:
lidar = xr.open_dataset('LiDAR/SGP_lidar.nc')
lidar

<xarray.Dataset>
Dimensions:               (height: 164, site: 5, time: 1872)
Coordinates:
  * height                (height) float32 90.93 116.9 ... 4.3e+03 4.326e+03
  * time                  (time) datetime64[ns] 2018-06-08 ... 2018-06-20T23:...
  * site                  (site) object 'C1' 'E32' 'E37' 'E39' 'E41'
Data variables: (12/31)
    scan_duration         (site, time) float32 ...
    elevation_angle       (site, time) float32 ...
    nbeams                (site, time) float64 ...
    u                     (site, time, height) float32 ...
    u_error               (site, time, height) float32 ...
    v                     (site, time, height) float32 ...
    ...                    ...
    met_alt               (site, time) float64 ...
    lat                   (site) float64 ...
    lon                   (site) float64 ...
    alt                   (site) float64 ...
    x                     (site) float64 ...
    y                     (site) float64 ...
Attributes: (12/17)
    process_version:       vap-dlprof_wind-0.8-0.el6
    command_line:          idl -R -n dlprof_wind -s sgp -f C1 -d 20180608
    dod_version:           dlprofwind4news-c1-1.0
    Conventions:           ARM-1.1
    site_id:               sgp
    platform_id:           dlprofwind4news
    ...                    ...
    dlat:                  36.60529
    dlon:                  -97.48658
    serial_number:         0116-107
    doi:                   DOI:10.5439/1178582
    doi_url:               http://dx.doi.org/10.5439/1178582
    history:               created by user dsmgr on machine ruby at 2018-06-0...

In [9]:
data = lidar.sel(site='C1').isel(height = slice(0, 24))
data['windspeed'] = data['wind_speed']
data['datetime'] = data['time']
data = data.rename(dict({'height': 'z', 'time': 't'}))
data['height'] = data['z']

In [10]:
jet_id = findLLJ.findLLJevents_xr_lidar(data)

In [11]:
jet_id.to_netcdf('LiDAR/SGP_jet_id.nc')